# Data transformation from EDF to numpy
This notebook should only be used as a complementary script to files at https://github.com/VlastaKoudelka/NUDZ-HYSCAN/tree/main/group and also interactive_annotations.py. The interactive annotations is in separate file because jupyter notebook does not save the annotated data.

A function of this notebook is to load an EEG data in the EDF format, extract data for specific conditions (Trust game and stare paradigm) and save them as a numpy data for further analysis. After processing the data with this script, processing with prepare_permutations.ipynb and graph_analysis.ipynb could be applied. 

## Load libraries


In [ ]:

import mne
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt #without this interactive plotting does not work

## Load the data

This cell loads data and events that were manualy annotated with an interactive_annotations.py

At first, you need to process the data of Stare paradigm in order to have a baseline for comparison with a Trust game conditions (This is a just a temporari solution)


In [ ]:
subj_n = 1

folder_pth = {'baseline_folder': 'C:/FEL/ING/diplomka/data/baseline_stare/kamdu/',
       'trust_game_folder': 'C:/FEL/ING/diplomka/data/TG/kamdu/'
}

file_tags = {'baseline': 'HYSCAN_GD_JS_2024-12-09_13-12-25_Segment_0',
             'trust_game': 'HYSCAN_GD_JS_2024-12-09_13-24-23_Segment_0',

}

# Load the data

raw_data = mne.io.read_raw_fif(folder_pth['baseline_folder']+file_tags['baseline'] + '_bad_annot.fif', preload = True)
events, event_id = mne.events_from_annotations(raw_data)

## Check the annotated data if necesary

# raw_data.plot()
# plt.show()


## Assign id's of the TG events

Each event has a specific indicator (an integer). In the cell below Trust game events are extracted

In [ ]:

tg_event_list = ['CueInv','ExStart','RepCue','SendInv','SendRep','ShowInv','ShowRep','ShowTot']
tg_events = {}

for true_idx, true_ev in enumerate(tg_event_list):
        
    for ev_idx, ev_val in enumerate(event_id):
        if true_ev in ev_val:
            tg_events[true_ev] = event_id[ev_val]

## Assign id's of the baseline events

Same as above but for baseline



In [ ]:

baseline_event_list = ['Cross', 'Stare']
baseline_events = {}


for true_idx, true_ev in enumerate(baseline_event_list):
        
    for ev_idx, ev_val in enumerate(event_id):
        if true_ev in ev_val:
            baseline_events[true_ev] = event_id[ev_val]


## These lines are for just to know the timing of wanted conditions
## RepCue - ShowInv = (89925-80551)/fs
##  ShowTot - ShowRep = (111002 - 101502) / fs

# fs = raw_data.info['sfreq']
# condA_delay = (89925 - 80551) / fs - 5 # four seconds behind RepCue
# condB_delay = (111002 - 101502) / fs - 5 #four seconds behind ShowTot 



## Filtering

Choose a specific frequency band for filtering. Optionaly you could also filter a power noise (50 Hz) and its harmonics

In [ ]:

# bandFreqs = [[1,4],[1,8],[8,12],[12,35],[35,45]]
# bandLabls = ['Delta','Theta','Alpha','Beta','Gamma']


power_freqs = np.linspace(50, 500, 10)
bandFreqs = [8,12]

## Notch filter is an optional step
# notch_filt = raw_data.copy().notch_filter(freqs = power_freqs)

baseline_alpha_filt = raw_data.copy().filter(bandFreqs[0],bandFreqs[1], fir_design='firwin')


## Get the data of interest

In this secntion you could choose the data for specific condition. The extracted files should be renamed in a format cxgy.npz, where x is a condition numbering and y is a group number.

Example: In order to get Trust game results computed with scripts prepare_permutations.ipynb and graph_analysis.ipyng you would need to save the baseline data from Stare experiment (sections with a cross) as c0g0.npz and trust game data from conditions 'RepCue' and 'ShowTot' as c1g0.npz and c2g0.npz.

Epoched TG data

In [ ]:
tmin = -4
tmax = 0
TG_condA = make_dyad_array(baseline_alpha_filt, tg_events['RepCue'], tmin, tmax)
TG_condB = make_dyad_array(baseline_alpha_filt, tg_events['ShowTot'], tmin, tmax)

np.savez(folder_pth['trust_game_folder'] + 'epoched_TG_ShIn_ReCu', TG_condA)
np.savez(folder_pth['trust_game_folder'] + 'epoched_TG_ShRep_ShTot', TG_condB)



Not setting metadata


15 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 15 events and 4097 original time points ...
2 bad epochs dropped
Not setting metadata
15 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 15 events and 4097 original time points ...
1 bad epochs dropped


Epoched baseline data

In [ ]:
Baseline_data = make_dyad_array(baseline_alpha_filt, baseline_events['Cross'], tmin, tmax)
np.savez(folder_pth['baseline_folder'] + 'epoched_cross', Baseline_data)

Not setting metadata
51 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 51 events and 4097 original time points ...
0 bad epochs dropped


## Functions

In [ ]:
def make_dyad_array(data, event, tmin, tmax):
    #This function creates an epochs for both subjects, then concatenates them to one matrix and converts to numpy array
    epoched_data = mne.Epochs(data, events, event_id=event, tmin=tmin, tmax=tmax, preload=True, baseline = None, reject_by_annotation=True)
    return epoched_data.get_data()